# DATA3888 Project Report: Holiday Planner

COVID C4

**Note:**

Our final submission for the report would need to be a ZIP file containing:
- this report;
- our data/ folder, containing all data files needed by this notebook and by our python analysis code files; and
- our python analysis code files (common.py, analytics.py, analytics_clustering.py, analytics_helper.py, analytics_helper_clustering.py).

## Executive summary

## Background

## Methods

Best approach might be to import all functions from common.py, analytics.py, analytics_clustering.py, analytics_helper.py and analytics_helper_clustering.py at the top of this report notebook, then include the contents of each of these files in the appendix.

**Need to add code commenting.**

## Results

See ClusteringKNNEvaluation.ipynb for clustering and KNN evaluation. Then there is also qualitative evaluation which I don't believe has code attached.

**Need to add code commenting.**

## Discussion

## Conclusion

## References

## Appendix

## Student contributions

### Eve Fernando

### Marie Montgomery

### Rayani Saha

### Serena Watson

### Stuart Toft

### Yan Liu